# 📒 Notebook : 08_Calculs_KPI

---

## 📝 Objectif

- Calculs et KPI's nécessaires à la visualtion BI.

---

## 👤 Auteur(s) / Contact

- SEKARI Inès — [ines.sekari@efrei.net]
- NKUIDA Malaïka - [malaika.nkuida@efrei.net]

---

## 🗓️ Versioning & Mise à jour

| Version | Date        | Modifications                          |
|---------|-------------|----------------------------------------|
| 1.0     | 2025-05-20  | Création du script des différents calculs       |


In [2]:
from pyspark.sql import SparkSession

# Démarrer la session Spark
spark = SparkSession.builder \
    .appName("LectureTablesHopital") \
    .getOrCreate()

# Table de faits
fact_consultation = spark.read.format("delta").load("Files/gold/fact_consultation")

# Dimensions
dim_patient = spark.read.format("delta").load("Files/gold/dim_patient")
dim_maladie = spark.read.format("delta").load("Files/gold/dim_maladie")
dim_motif_admission = spark.read.format("delta").load("Files/gold/dim_motif_admission")
dim_medicament = spark.read.format("delta").load("Files/gold/dim_medicament")
dim_hopital = spark.read.format("delta").load("Files/gold/dim_hopital")
dim_temps = spark.read.format("delta").load("Files/gold/dim_temps")

# Un affichage rapide pour vérifier
display(fact_consultation.head(5))
display(dim_patient.head(5))


StatementMeta(, 86185bd4-f6cc-4191-852a-cf9d164a0436, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, da093ca5-91d9-416e-91a6-3a8e8eebfd82)

SynapseWidget(Synapse.DataFrame, e8ae0052-7931-4e2d-9ecb-8377ff586f48)

In [4]:
#1. KPI de volumétrie et d’accès aux soins

#a. Nombre de passages aux urgences par période (jour/mois/année)
kpi_passages_par_jour = fact_consultation.groupBy("date_id").count().withColumnRenamed("count", "nb_passages")

display(kpi_passages_par_jour.head(5))


fact_consultation = fact_consultation.join(
    kpi_passages_par_jour,
    on="date_id",
    how="left"
)

display(fact_consultation.head(5))



from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col, datediff, lag

#c. Répartition des arrivées par mode d’admission
kpi_arrival_mode = dim_patient.groupBy("arrivalmode").count().orderBy("count", ascending=False)

display(kpi_arrival_mode.head(5))

dim_patient = dim_patient.join(
    kpi_arrival_mode,
    on="arrivalmode",
    how="left"
)

display(dim_patient.head(5))


StatementMeta(, 86185bd4-f6cc-4191-852a-cf9d164a0436, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4fb81355-017e-43d1-bb3b-6d6df4af2b28)

SynapseWidget(Synapse.DataFrame, f11f84ed-e39a-4f7b-a163-9ef7bcfd4251)

SynapseWidget(Synapse.DataFrame, 69304783-d2f6-4348-b8cd-f5e1ff1094de)

SynapseWidget(Synapse.DataFrame, ec3b0c95-bbe2-463e-abcb-341059bb1bd5)

In [5]:
#2. KPI de profil patient et disparités
#a. Répartition par genre, tranches d’âges
fact_consultation = fact_consultation.withColumn("tranche_age", 
  F.when(col("age") < 18, "0-17")
   .when((col("age") >= 18) & (col("age") < 40), "18-39")
   .when((col("age") >= 40) & (col("age") < 65), "40-64")
   .otherwise("65+")
)
kpi_age_gender = fact_consultation.groupBy("tranche_age", "gender").count()

fact_consultation = fact_consultation.join(
    kpi_age_gender,
    on=["tranche_age", "gender"],
    how="left"
)

display(fact_consultation.head(5))


#b. Analyse des profils sociaux (emploi, assurance, etc.)
kpi_assurance = dim_patient.groupBy("insurance_status").count().orderBy("count", ascending=False)
kpi_emploi = dim_patient.groupBy("employstatus").count()

display(kpi_emploi.head(5))
display(kpi_assurance.head(5))

# Jointure sur insurance_status
dim_patient = dim_patient.join(
    kpi_assurance,
    on="insurance_status",
    how="left"
)

# Jointure sur employstatus
dim_patient = dim_patient.join(
    kpi_emploi,
    on="employstatus",
    how="left"
)

display(dim_patient.head(5))


StatementMeta(, 86185bd4-f6cc-4191-852a-cf9d164a0436, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f6838830-4347-4eac-987a-d9a14ef71cd1)

SynapseWidget(Synapse.DataFrame, a194dac8-5b09-480b-bd91-f27dff0a3efe)

SynapseWidget(Synapse.DataFrame, b9064ccb-87cd-483b-9edd-999e95a67792)

SynapseWidget(Synapse.DataFrame, e1023594-8cb4-4a5d-862f-89d8d22752d1)

In [ ]:
#3. KPI par pathologie / motifs médicaux / saisonnalité
#a. Evolution des principaux motifs d’admission
from pyspark.sql.functions import explode

# Admettons que "motifs" est un array/list des motifs d’un passage
motifs_explosed = fact_consultation.withColumn("motif_admission", explode("liste_motif_admission"))
kpi_motif_temps = motifs_explosed.groupBy("motif_admission", "date_id").count()

display(kpi_motif_temps.head(5))


#b. Classement des pathologies principales et leur évolution
maladies_explosed = fact_consultation.withColumn("maladie", explode("liste_maladie"))
kpi_top_maladies = maladies_explosed.groupBy("maladie").count().orderBy("count", ascending=False)

display(kpi_top_maladies.head(5))



#c. Liens entre motif d’admission/diagnostic/médicament utilisé
motif_medic = fact_consultation \
    .withColumn("motif_admission", explode("liste_motif_admission")) \
    .withColumn("medicament", explode("liste_medicament")) \
    .groupBy("motif_admission", "medicament") \
    .count()

display(motif_medic.head(5))


#d. Analyse du recours à la chirurgie
kpi_chirurgie = fact_consultation.groupBy("hopital_nom")\
    .agg(F.sum("nb_chirurgies").alias("chirurgies_totales"),
         F.avg("nb_chirurgies").alias("chirurgies_moyennes"))

display(kpi_chirurgie.head(5))



StatementMeta(, 15029b16-8ea7-4ebc-937c-c43aa9a1b5e0, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3362513b-e7dc-44b0-b587-6aad5e88d2c8)

SynapseWidget(Synapse.DataFrame, bd76484d-010d-4fc2-8d36-ca2cfc517767)

SynapseWidget(Synapse.DataFrame, d6572081-f9b4-4395-a65d-e490f0971cb0)

SynapseWidget(Synapse.DataFrame, 16f06d37-124a-4ba3-b9c3-c281cb00265c)

In [ ]:
#4. KPI de sortie et impact du passage
#a. Taux d’hospitalisation après passage aux urgences
kpi_hospi = fact_consultation.groupBy("disposition").count()
kpi_hospi = kpi_hospi.withColumn("taux", col("count")/F.sum("count").over(Window.partitionBy()))

display(kpi_hospi.head(5))



StatementMeta(, 15029b16-8ea7-4ebc-937c-c43aa9a1b5e0, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6457cc0f-b656-4984-982c-8602acb48143)

In [ ]:
#5. Analyse transverse/socio-médicale intéressantes

#Analyse patient “complexes” : ceux avec ≥ 3 pathologies
# Compte le nombre de pathologies par patient sur toutes ses visites
from pyspark.sql.functions import split, size

fact_complexe = fact_consultation \
    .withColumn("nb_maladies", size("liste_maladie")) \

# Filtre patients “complexes”
patients_complexes = fact_complexe \
    .filter(F.col("nb_maladies") >= 3) \
    .select("patient_id", "nb_maladies").distinct()

display(patients_complexes.head(5))


StatementMeta(, 15029b16-8ea7-4ebc-937c-c43aa9a1b5e0, 39, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e55f0b78-3784-4694-9778-a9e4ded77bb3)